In [1]:
from helper import Helper
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time
import matplotlib.pyplot as plt

In [2]:
h = Helper()

train_it, validation_it, test_it = h.construct_data_generator_w_validation(batch_size=10)

Found 22050 images belonging to 3 classes.
Found 3150 images belonging to 3 classes.
Found 6300 images belonging to 3 classes.


In [3]:
vgg19 = VGG19(include_top=False, weights=None, input_shape=(224,224,3))
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
x = vgg19.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)

pred = Dense(3, activation='softmax')(x)

model = Model(inputs=vgg19.input, outputs=pred, name='cnn_topomap_classification_model_vgg19')
model.summary()

Model: "cnn_topomap_classification_model_vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [6]:
opt = RMSprop(learning_rate=0.00001, momentum=0.9)
loss = 'categorical_crossentropy'

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [7]:
tblog_path = h.logs_directory.format(time())
tb_log = TensorBoard(log_dir = tblog_path, profile_batch=0)

ch = ModelCheckpoint('models/vgg19/cnn_topomap_classification_model_vgg19.h5', monitor='val_accuracy', 
                     mode='max', save_best_only=True)

es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=5)

In [8]:
r = model.fit_generator(train_it, epochs=100, callbacks=[ch, es, tb_log], validation_data=validation_it)

Epoch 1/100
2205/2205 [==============================] - 285s 129ms/step - loss: 1.0987 - accuracy: 0.3285 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 2/100
2205/2205 [==============================] - 281s 127ms/step - loss: 1.0987 - accuracy: 0.3298 - val_loss: 1.0987 - val_accuracy: 0.3333
Epoch 3/100
2205/2205 [==============================] - 281s 127ms/step - loss: 1.0987 - accuracy: 0.3365 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/100
2205/2205 [==============================] - 281s 128ms/step - loss: 1.0987 - accuracy: 0.3253 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/100
2205/2205 [==============================] - 281s 127ms/step - loss: 1.0987 - accuracy: 0.3305 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 6/100
2205/2205 [==============================] - 280s 127ms/step - loss: 1.0987 - accuracy: 0.3323 - val_loss: 1.0986 - val_accuracy: 0.3333


In [ ]:
evaluate = model.evaluate_generator(test_it)

In [ ]:
y_prob = model.predict_generator(test_it)

In [ ]:
h.save(model, r, evaluate, y_prob, 'cnn_topomap_classification_model_vgg19')

In [ ]:
model_ch = h.load('models/vgg19/cnn_topomap_classification_model_vgg19.h5')

In [ ]:
e_ch = model_ch.evaluate_generator(test_it)

In [ ]:
e_ch